In [2]:
!pip install jovian opendatasets matplotlib seaborn xgboost --upgrade --quiet


In [3]:
# Import packages
import jovian
import opendatasets as od
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

## Downloading the Data
The dataset is obtained from kaggle. The dataset contains information on historic trades for several cryptoassets, such as Bitcoin and Ethereum.

In [4]:
data_url = 'https://www.kaggle.com/c/g-research-crypto-forecasting/data'
od.download(data_url)

In [5]:
data_dir = './g-research-crypto-forecasting'


In [6]:
os.listdir(data_dir)


## Data Preparation and Cleaning
In this section, train.csv will be read in. The training set has the following variables

timestamp - A timestamp for the minute covered by the row.

Asset_ID - An ID code for the cryptoasset.

Count - The number of trades that took place this minute.

Open - The USD price at the beginning of the minute.

High - The highest USD price during the minute.

Low - The lowest USD price during the minute.

Close - The USD price at the end of the minute.

Volume - The number of cryptoasset units traded during the minute.

VWAP - The volume weighted average price for the minute.

Target - 15 minute residualized returns.

The following variables will be created based on the timestamp column.

hour is the hour of the day

weekday is the weekday of the week

Categorical variables (Asset_ID, hour, and weekday) will be on hot encoded.

Numeric variables (Count, Open, High, Low, Close, Volume, and VWAP) will be scaled to the value of 0 to 1 for each of the Asset_ID.

## Read in Dataset

In [8]:
train_df = pd.read_csv('./g-research-crypto-forecasting/train.csv')
asset_details_df = pd.read_csv('./g-research-crypto-forecasting/asset_details.csv')
test_df = pd.read_csv('./g-research-crypto-forecasting/example_test.csv')

In [9]:
train_df.shape


The training data has 24 million records and 10 columns

In order to reduce the data for lighter manipulation we take a subset up to November 2020

In [10]:
# Convert timestamp to date time
train_df['timestamp'] = train_df.timestamp.astype('datetime64[s]')
train_df = train_df[train_df.timestamp.dt.year==2020]
train_df = train_df[train_df.timestamp.dt.month==11]
train_df.shape

The reduced training set has 598769 records.

## Handeling Missing Values
Check missing values

In [11]:
train_df.isna().sum()


There are 8814 records with missing data available

In [12]:
train_df.isin([np.nan, np.inf, -np.inf]).sum()


There is no infinity value in the data

The following code will drop the missing values and infinity values.

In [13]:
train_df.dropna(inplace=True)
train_df = train_df[np.isfinite(train_df).all(1)]

In [14]:
train_df.describe()


## Feature Engineering
The following code converts timestamp to date time type and creates variables hour and weekday based on the value of column timestamp.

In [15]:
# Create hour variable
train_df['hour'] = train_df.timestamp.dt.hour

# Create weekday variable
train_df['weekday'] = train_df.timestamp.dt.weekday

### Encode Categorical Variables
This section will convert variables Asset_ID, hour and weekday to categorical variables and create one hot encoder for categorical variables

In [16]:
# Convert Asset_ID, hour and weekday to categorical
train_df['Asset_ID'] = train_df.Asset_ID.astype('category')
train_df['hour'] = train_df.hour.astype('category')
train_df['weekday'] = train_df.weekday.astype('category')

In [17]:
# Set up Encoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# fit onehotcoder
encoder.fit(train_df[['Asset_ID','hour','weekday']])

In [18]:
# get new encoded cols names
encoded_cols = list(encoder.get_feature_names(['Asset_ID','hour','weekday']))

# replace categorical variables with one hot encoder
train_df[encoded_cols] = encoder.transform(train_df[['Asset_ID','hour','weekday']])

### Scale Numeric Variables
This section will scale numeric variables Count, Open, High, Low, Close, Volume, and VWAP to range from 0 to 1.

In [19]:
scaler = MinMaxScaler()

num_cols = ['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']

scaler.fit(train_df[num_cols])
train_df[num_cols] = scaler.transform(train_df[num_cols])

## Time Series Plots
The following code get the time range for each of the Asset_ID.

In [20]:
train_df.groupby(["Asset_ID"]).agg({'timestamp': [np.min,np.max]})


The date range is slightly different for each of the asset.



In [21]:
plt.figure(figsize=(50, 25))
sns.lineplot(data=train_df, x="timestamp", y="Target", hue="Asset_ID")

The Target variable has very different patterns for each of the asset of interest.

#### Correlation Matrix
The following code provide correlation matrix for each of the asset.

In [22]:
corr_matrix = train_df[['Count','Open','High','Low','Close','Volume','VWAP','Target']].corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix,annot=True, cmap='Blues');

The Open, High, Low, Close, and VWAP are highly correlated.

### Training and Testing Split

Use data from November 1st - November 20th as training set, use the rest data in November as validation set. The testing set is provided in example_test.csv file.

In [23]:
train_set = train_df[train_df.timestamp.dt.day <= 20]
val_set = train_df[train_df.timestamp.dt.day > 20]

In [24]:
train_set.shape


In [25]:
val_set.shape


In [26]:
train_set.Asset_ID.value_counts()


In [27]:
val_set.Asset_ID.value_counts()


### Random Forest

This section build random forest to predict the Target variable.

In [28]:
# Create the model
rf1 = RandomForestRegressor(random_state=5, n_jobs = -1)

In [29]:
# Fit the model
input_cols = num_cols + encoded_cols
target_col = 'Target'

rf1.fit(train_set[input_cols], train_set[target_col])

In [30]:
# Model Evaluation
tree_train_preds = rf1.predict(train_set[input_cols])
tree_val_preds = rf1.predict(val_set[input_cols])

tree_train_rmse = mean_squared_error(train_set[target_col], tree_train_preds, squared=False)
tree_val_rmse = mean_squared_error(val_set[target_col], tree_val_preds, squared=False)

print("Train Error is ", tree_train_rmse)
print("Validation Error is ", tree_val_rmse)

In [31]:
# functions for hyperparameter tunning
def test_params(**params):
    model = RandomForestRegressor(random_state=5, n_jobs=-1, **params).fit(train_set[input_cols], train_set[target_col])
    train_rmse = mean_squared_error(model.predict(train_set[input_cols]), train_set[target_col], squared=False)
    val_rmse = mean_squared_error(model.predict(val_set[input_cols]), val_set[target_col], squared=False)
    return train_rmse, val_rmse

def test_param_and_plot(param_name, param_values):
    train_errors, val_errors = [], [] 
    for value in param_values:
        params = {param_name: value}
        train_rmse, val_rmse = test_params(**params)
        train_errors.append(train_rmse)
        val_errors.append(val_rmse)
    plt.figure(figsize=(10,6))
    plt.title('Overfitting curve: ' + param_name)
    plt.plot(param_values, train_errors, 'b-o')
    plt.plot(param_values, val_errors, 'r-o')
    plt.xlabel(param_name)
    plt.ylabel('RMSE')
    plt.legend(['Training', 'Validation'])

In [32]:
# max_depth
test_param_and_plot('max_depth', [5, 10, 15, 20, 25, 30, 35])